In [66]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

# 키워드 리스트
keywords = [
    "공포", "집값 폭락"
]

# keywords = [
#     "공포", "집값 폭락", "부동산 거품", "아파트 하락", "금리 인상", "매매 침체",
#     "부동산 호황", "매수 타이밍", "아파트 급등", "부동산 투자", "집값 상승"
# ]
# 날짜 설정
start_date = "2025-01-01"
end_date = "2025-01-10"
# 네이버 뉴스 URL
naver_url = "https://search.naver.com/search.naver?where=news"
# 크롤링 결과 저장 리스트
results = []

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저를 띄우지 않고 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

def calculate_quarter(date_str):
    """날짜 문자열로부터 분기 계산"""
    try:
        date = pd.to_datetime(date_str, errors="coerce")
        year = date.year % 100  # 두 자리 연도로 변환
        quarter = (date.month - 1) // 3 + 1
        return f"{year}년{quarter}분기"
    except Exception:
        return "날짜 오류"
    
def get_articles(keyword):
    """키워드로 뉴스 크롤링"""
    global results
    driver.get(naver_url)
    time.sleep(2)
    
    # 검색창에 키워드 입력
    try:
        print("[DEBUG] 검색창 로드 대기 중...")
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "query"))
        )
        print("[DEBUG] 검색창 로드 완료!")
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
    except Exception as e:
        print(f"[DEBUG] 검색창 처리 중 에러: {e}")
        return

    try:
        print("[DEBUG] 옵션 버튼 찾는 중...")
        option_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.btn_option._search_option_open_btn"))
        )
        print("[DEBUG] 옵션 버튼 찾기 완료!")
        
        # 스크롤하여 버튼이 화면 중앙으로 오도록 조정
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", option_button)
        print("[DEBUG] 옵션 버튼 스크롤 완료!")
        
        # ActionChains로 클릭 강제 실행
        action = ActionChains(driver)
        action.move_to_element(option_button).click().perform()
        print("[DEBUG] 옵션 버튼 클릭 완료!")
        time.sleep(1)
    except Exception as e:
        print("[ERROR] 옵션 버튼 처리 중 에러 발생:", e)

    try:
        print("[DEBUG] 직접 입력 버튼 찾는 중...")
        # "직접 입력" 버튼 로드 대기
        direct_input_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.txt.txt_option._calendar_select_trigger"))
        )
        print("[ERROR] 직접 입력 버튼 찾기 완료!")
        
        # 스크롤하여 버튼이 화면 중앙으로 오도록 조정
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", direct_input_button)
        print("[DEBUG] 직접 입력 버튼 스크롤 완료!")
        
        # ActionChains로 클릭 강제 실행
        action = ActionChains(driver)
        action.move_to_element(direct_input_button).click().perform()
        print("[DEBUG] 직접 입력 버튼 클릭 완료!")
        time.sleep(1)
    except Exception as e:
        print("[ERROR] 직접 입력 버튼 처리 중 에러 발생:", e)



    # 날짜 필터 적용
    try:
        # 시작 날짜 설정
        print("[DEBUG] 시작 날짜 설정 중...")
        start_date_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a._start_trigger"))
        )
        start_date_button.click()
        time.sleep(1)

        start_year = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link' and text()='2025']"))
        )
        start_year.click()
        time.sleep(1)

        start_month = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link' and text()='1']"))
        )
        start_month.click()
        time.sleep(1)

        start_day = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link' and text()='19']"))
        )
        start_day.click()
        time.sleep(1)

        # 끝 날짜 설정
        print("[DEBUG] 끝 날짜 설정 중...")
        end_date_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a._end_trigger"))
        )
        end_date_button.click()
        time.sleep(1)

        end_year = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link' and text()='2025']"))
        )
        end_year.click()
        time.sleep(1)

        end_month = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link' and text()='1']"))
        )
        end_month.click()
        time.sleep(1)

        end_day = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link' and text()='19']"))
        )
        end_day.click()
        time.sleep(1)

        # "적용" 버튼 클릭
        print("[DEBUG] 적용 버튼 클릭 중...")
        apply_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '적용')]"))
        )
        apply_button.click()
        print("[DEBUG] 날짜 필터 설정 완료!")
        time.sleep(2)
    except Exception as e:
        print(f"[ERROR] 날짜 필터 적용 중 에러: {e}")
        return

    # 뉴스 기사 크롤링
    page = 1
    while True:
        try:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            articles = soup.select("ul.list_news > li")
            
            if not articles:
                break
            
            for article in articles:
                try:
                    title = article.select_one("a.news_tit").get_text()
                    url = article.select_one("a.news_tit")["href"]
                    content = article.select_one("div.news_dsc").get_text() if article.select_one("div.news_dsc") else "내용 없음"
                    press = article.select_one("a.info.press").get_text() if article.select_one("a.info.press") else "출처 없음"
                    date = article.select_one("span.info").get_text() if article.select_one("span.info") else "날짜 없음"
                    quarter = calculate_quarter(date)
                    
                    results.append({
                        "키워드": keyword,
                        "제목": title,
                        "내용": content,
                        "URL": url,
                        "작성 시간": date,
                        "분기": quarter,
                        "기자 이름": press
                    })
                except Exception as e:
                    print("[ERROR] 기사 처리 중 에러:", e)
            
            # 다음 페이지로 이동
            next_button = soup.select_one("a.btn_next")
            if next_button and "href" in next_button.attrs:
                driver.get("https://search.naver.com" + next_button["href"])
                time.sleep(2)
            else:
                break
            page += 1
        except Exception as e:
            print(f"[ERROR] 페이지 크롤링 중 에러: {e}")
            break
# 키워드별로 기사 수집
for keyword in keywords:
    print(f"[INFO] '{keyword}' 키워드로 크롤링 중")
    get_articles(keyword)

# Selenium 드라이버 종료
driver.quit()
# CSV 파일로 저장
output_file = "naver_news_crawling.csv"
df = pd.DataFrame(results)
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"[INFO] 크롤링 완료. 결과가 {output_file}에 저장되었습니다.")